In [1]:
import sagemaker
sess=sagemaker.Session(default_bucket="sagemaker-us-east-1-470086202700")
role= sagemaker.get_execution_role()
account =sess.boto_session.client('sts').get_caller_identity()['Account']
region= sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
print(sess.default_bucket())

sagemaker-us-east-1-470086202700


In [3]:
print(role)

arn:aws:iam::470086202700:role/sagemaker-developers


## Segmentation Docker build

In [4]:
!cat Dockerfile

# Base image from NVIDIA for PyTorch with CUDA support
FROM nvcr.io/nvidia/pytorch:23.12-py3 as build

# Install required dependencies
RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y install tzdata && \
    apt install -y zip libgl1-mesa-glx netbase libopencv-dev libopenblas-dev nginx && \
    apt clean && rm -rf /var/lib/apt/lists/*

# Install pip and specific versions of torch and torchvision
RUN curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py && \
    python get-pip.py && \
    python -m pip install pip==24.2 --no-cache-dir

# Set working directory and copy scripts
WORKDIR /opt/program
COPY ./scripts /opt/program

# Install specific torch and torchvision versions
RUN pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 --no-cache-dir

# Install other Python dependencies
RUN pip install -r requirements.txt --no-cache-dir

# Set environment variables
ENV PYTHONUNBUFFERED=TRUE \
    PYTHONDONTWRITEBYTECODE=

In [5]:
repo_name="detectron2_segmentation_bt"
base_job_name=image_tag=repo_name
%env dockerfile Dockerfile

env: dockerfile=Dockerfile


### Below cells sets up enironment variables build dokcer image and push to ECR If the image is already in ECR skip these steps

In [6]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: account=470086202700
env: region=us-east-1
env: repo_name=detectron2_segmentation_bt
env: image_tag=detectron2_segmentation_bt


In [7]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


In [10]:
!docker build -t $image_tag -f $dockerfile .

Sending build context to Docker daemon   41.1MB
Step 1/10 : FROM nvcr.io/nvidia/pytorch:23.12-py3 as build
23.12-py3: Pulling from nvidia/pytorch

17c0bd28: Pulling fs layer 
b8c4201a: Pulling fs layer 
b700ef54: Pulling fs layer 
e45c417e: Pulling fs layer 
aa4df5f4: Pulling fs layer 
a8efd95b: Pulling fs layer 
cea7e0d5: Pulling fs layer 
8c4cfff7: Pulling fs layer 
61f07a8d: Pulling fs layer 
9f292aaf: Pulling fs layer 
1100b2de: Pulling fs layer 
f0db8a6e: Pulling fs layer 
b4f11558: Pulling fs layer 
6602088c: Pulling fs layer 
8efd95b: Waiting fs layer 
8c4cfff7: Waiting fs layer 
ea7e0d5: Waiting fs layer 
b4f11558: Waiting fs layer 
881250ad: Pulling fs layer 
6ff49dd8: Waiting fs layer 
bc53b342: Pulling fs layer 
a5c0eb02: Pulling fs layer 
e451b1e1: Pulling fs layer 
b329b8d2: Pulling fs layer 
b329b8d2: Waiting fs layer 
a1947776: Pulling fs layer 
9294e66a: Pulling fs layer 
e742d5d4: Pulling fs layer 
fdbf80b7: Waiting fs layer 
505ff330: Pulling fs layer 
f5d886b6: Pulli

ea7e0d5: Extracting  11.53kB/11.53kBB49ADownloading     116B/116BExtracting  981.5MB/2.958GB

5d886b6: Extracting  22.71MB/27.86MB2AExtracting     116B/116BExtracting  1.616GB/2.287GB

8321cc6f: Pull complete  503B/503B9kBExtracting  291.9MB/1.781GBExtracting  1.311MB/3.7MBDigest: sha256:da3d1b690b9dca1fbf9beb3506120a63479e0cf1dc69c9256055125460eb44f7
Status: Downloaded newer image for nvcr.io/nvidia/pytorch:23.12-py3
 ---> 1cff6923bda0
Step 2/10 : RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y install tzdata &&     apt install -y zip libgl1-mesa-glx netbase libopencv-dev libopenblas-dev nginx &&     apt clean && rm -rf /var/lib/apt/lists/*
 ---> Running in 635646af3e3d


Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1149 kB]
Get:7 http://archive.ubuntu.c

The following NEW packages will be installed:
  adwaita-icon-theme at-spi2-core dbus dbus-user-session
  dconf-gsettings-backend dconf-service dmsetup fontconfig fontconfig-config
  fonts-dejavu-core gdal-data gir1.2-glib-2.0 gsettings-desktop-schemas
  gstreamer1.0-plugins-base gtk-update-icon-cache hicolor-icon-theme
  humanity-icon-theme i965-va-driver intel-media-va-driver iproute2 iso-codes
  libaacs0 libaom3 libapparmor1 libarchive13 libargon2-1 libarmadillo10
  libarpack2 libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data libatm1
  libatspi2.0-0 libavahi-client3 libavahi-common-data libavahi-common3
  libavcodec-dev libavcodec58 libavformat-dev libavformat58 libavutil-dev
  libavutil56 libbdplus0 libblosc1 libbluray2 libbpf0 libcairo-gobject2
  libcairo2 libcap2-bin libcdparanoia0 libcfitsio9 libcharls2 libchromaprint1
  libcodec2-1.0 libcolord2 libcryptsetup12 libcups2 libdatrie1 libdav1d5
  libdbus-1-3 libdc1394-25 libdc1394-dev libdconf1 libde265-0 libdeflate-dev
  libdeflate0 lib

...

Setting up libopenjp2-7:amd64 (2.4.0-6) ...
Setting up libx11-6:amd64 (2:1.7.5-1ubuntu0.3) ...
Setting up libharfbuzz0b:amd64 (2.7.4-1ubuntu3.1) ...
Setting up libthai-data (0.1.29-1build1) ...
Setting up libgirepository-1.0-1:amd64 (1.72.0-1) ...
Setting up netbase (6.3) ...
Setting up libvpx7:amd64 (1.11.0-2ubuntu2.3) ...
Setting up libatk1.0-0:amd64 (2.36.0-3build1) ...
Setting up libtiff5:amd64 (4.3.0-6ubuntu0.10) ...
Setting up libwayland-egl1:amd64 (1.20.0-1ubuntu0.1) ...
Setting up libusb-1.0-0:amd64 (2:1.0.25-1ubuntu2) ...
Setting up libtbb-dev:amd64 (2021.5.0-7ubuntu2) ...
Setting up libgphoto2-port12:amd64 (2.5.27-1build2) ...
Setting up libpsm-infinipath1 (3.3+20.604758e7-6.1) ...
update-alternatives: using /usr/lib/libpsm1/libpsm_infinipath.so.1.16 to provide /usr/lib/x86_64-linux-gnu/libpsm_infinipath.so.1 (libpsm_infinipath.so.1) in auto mode
Setting up libdav1d5:amd64 (0.9.2-1) ...
Setting up libmfx1:amd64 (22.3.0-1) ...
Setting up libnginx-mod-http-geoip2 (1.18.0-6ubunt

Setting up libxslt1.1:amd64 (1.1.34-4ubuntu0.22.04.1) ...
Setting up libgdk-pixbuf-2.0-0:amd64 (2.42.8+dfsg-1ubuntu0.3) ...
Setting up libswscale5:amd64 (7:4.4.2-0ubuntu0.22.04.1) ...
Setting up libcairo-gobject2:amd64 (1.16.0-5ubuntu2) ...
Setting up mesa-va-drivers:amd64 (23.2.1-1ubuntu3.1~22.04.2) ...
Setting up libpangoft2-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Setting up libbluray2:amd64 (1:1.3.1-1) ...
Setting up i965-va-driver:amd64 (2.4.1+dfsg1-1) ...
Setting up libpangocairo-1.0-0:amd64 (1.50.6+ds-2ubuntu1) ...
Setting up libavutil-dev:amd64 (7:4.4.2-0ubuntu0.22.04.1) ...
Setting up networkd-dispatcher (2.1-2ubuntu0.22.04.2) ...
Created symlink /etc/systemd/system/multi-user.target.wants/networkd-dispatcher.service → /lib/systemd/system/networkd-dispatcher.service.
Setting up libopencv-shape-dev:amd64 (4.5.4+dfsg-9ubuntu4) ...
Setting up libgl1-amber-dri:amd64 (21.3.9-0ubuntu1~22.04.1) ...
Setting up mesa-vdpau-drivers:amd64 (23.2.1-1ubuntu3.1~22.04.2) ...
Setting up libopencv-i

  Attempting uninstall: torch
    Found existing installation: torch 2.2.0a0+81ea7a4
    Uninstalling torch-2.2.0a0+81ea7a4:
      Successfully uninstalled torch-2.2.0a0+81ea7a4
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.0a0
    Uninstalling torchvision-0.17.0a0:
      Successfully uninstalled torchvision-0.17.0a0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-tensorrt 2.2.0a0 requires torch<2.3.0,>=2.1.0.dev, but you have torch 1.12.1+cu113 which is incompatible.
torchdata 0.7.0a0 requires torch>=2, but you have torch 1.12.1+cu113 which is incompatible.
torchtext 0.17.0a0 requires torch==2.2.0a0+81ea7a4, but you have torch 1.12.1+cu113 which is incompatible.
Removing intermediate container 91c4925eb8ca
 ---> 7f298e0563cd
Step 7/10 : RUN pip install -r requirements.txt --no-cache-dir
 ---> Running in 5df0549a5ba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 271.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 318.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.1/617.1 kB 548.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 545.2 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=292621233b3998f2a17a8a567da54e8c3812c556c137d058511441d7bcf81ba8
  Stored in directory: /tmp/pip-ephem-wheel-cache-mgt8q9_h/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for matplotlib: filename=matplotlib-3.3.4-cp310-cp310-linux_x86_64.whl size=11685733 sha256=55a306fcc7b610337995c04dc43b7af5639ca1f4de117ff124e0f14dda2fd6f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mgt8q9_h/wheels/38/c6/49/eaba6d234887d98d9c85185e2a90bd7bb77934e85eefaf317e
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=54094

  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.8.2
    Uninstalling matplotlib-3.8.2:
      Successfully uninstalled matplotlib-3.8.2
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0+nv0.8.0
    Uninstalling pycocotools-2.0+nv0.8.0:
      Successfully uninstalled pycocotools-2.0+nv0.8.0
Removing intermediate container 5df0549a5bad
 ---> d932e83217c0
Step 8/10 : ENV PYTHONUNBUFFERED=TRUE     PYTHONDONTWRITEBYTECODE=TRUE     PATH="/opt/program:${PATH}"     MODEL_SERVER_WORKERS=1
 ---> Running in 8b870ae028fd
Removing intermediate container 8b870ae028fd
 ---> 175162a0a5e8
Step 9/10 : RUN chmod +x /opt/program/serve
 ---> Running in fd29b1100d89
Removing intermediate container fd29b1100d89
 ---> 8bc02d225c48
Step 10/10 : ENTRYPOINT ["./serve"]
 ---> Running in 4085faadbeac

In [11]:
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [12]:
 !docker images

REPOSITORY                                                                TAG         IMAGE ID       CREATED                  SIZE
470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation_bt   latest      afda172a4b6f   Less than a second ago   26.7GB
detectron2_segmentation_bt                                                latest      afda172a4b6f   Less than a second ago   26.7GB
nvcr.io/nvidia/pytorch                                                    23.12-py3   1cff6923bda0   9 months ago             21.9GB


In [13]:
 !docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

The push refers to repository [470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation_bt]

0130f7b1: Preparing 
873f69c2: Preparing 
5b98d25c: Preparing 
dd8dd94a: Preparing 
75107be5: Preparing 
8134cd52: Preparing 
ab5d96a8: Preparing 
47f57c32: Preparing 
4b1505f6: Preparing 
220ff626: Preparing 
3a512c86: Preparing 
6890015a: Preparing 
d41b5bf7: Preparing 
8134cd52: Waiting g 
47f57c32: Waiting g 
4b1505f6: Waiting g 
220ff626: Waiting g 
db49e47c: Preparing 
d2b43e5a: Preparing 
a0126c98: Preparing 
bf18a086: Preparing 
71b4858c: Preparing 
cd9dc845: Preparing 
7dc31822: Preparing 
8b0ad8c5: Preparing 
4f6d038: Waiting g 
b5c60d0: Waiting g 
3f573fd: Waiting g 
8864767: Waiting g 
cd9dc845: Waiting g 
b49e47c: Waiting g 
0126c98: Waiting g 
dc31822: Waiting g 
1bbe2136: Preparing 
1b4858c: Waiting g 
6e8e18e: Waiting g 
4dae3b9: Waiting g 
441e172: Waiting g 
d62cb9b: Waiting g 
5eb74a10: Waiting g 
ecdb7186: Waiting g 
a256c52f: Waiting g 
b1d479d5: Preparing 
2f12

b98d25c: Pushing  1.285GB/3.532GBPushing     512B/10.74kBPushing  415.2MB/3.532GB

b98d25c: Pushed   3.541GB/3.532GBlatest: digest: sha256:06e6711472f3a10f1966ee38d4a870ad497724338f319e3263a09f45d421b4a0 size: 12329
